# FER-2013 Dataset


### **FER-2013 avec ResNet18**

**📚 2. Importation des bibliothèques**

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import kagglehub

**📥 3. Définition du dataset FER-2013**

In [2]:
class FER2013Dataset(Dataset):
    def __init__(self, csv_path, usage="Training", transform=None):
        self.data = pd.read_csv(csv_path)
        self.data = self.data[self.data["Usage"] == usage]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        emotion = int(self.data.iloc[idx]["emotion"])
        img = np.fromstring(self.data.iloc[idx]["pixels"], sep=" ", dtype=np.uint8).reshape(48, 48)
        img = Image.fromarray(img).convert("L")
        if self.transform:
            img = self.transform(img)
        return img, emotion

**🧰 4. Prétraitement des images**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Adapter à ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

**📂 5. Chargement des données**

In [ ]:
path = kagglehub.dataset_download("msambare/fer2013")

train_dataset = FER2013Dataset(path, usage="Training", transform=transform)
val_dataset = FER2013Dataset(path, usage="PublicTest", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

100%|██████████| 60.3M/60.3M [00:03<00:00, 16.9MB/s]

Extracting files...


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\natha\\.cache\\kagglehub\\datasets\\msambare\\fer2013\\versions\\1'

**🧠 6. Création du modèle ResNet18**

In [7]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 7)  # 7 classes d’émotions

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

c:\Users\natha\Desktop\Travail\TPS\2A\TPE\TPE_ISSD\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\natha\Desktop\Travail\TPS\2A\TPE\TPE_ISSD\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\natha/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:04<00:00, 11.1MB/s]


**⚙️ 7. Fonction d'entraînement et d'évaluation**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
    acc = correct / len(loader.dataset)
    return running_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
    acc = correct / len(loader.dataset)
    return val_loss / len(loader), acc

**🚀 8. Boucle d'entraînement**

In [ ]:
num_epochs = 10
train_acc_list, val_acc_list = [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    print(f"Époque {epoch+1} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


**📈 9. Courbe de performance**

In [ ]:
plt.plot(train_acc_list, label="Train Accuracy")
plt.plot(val_acc_list, label="Validation Accuracy")
plt.xlabel("Époque")
plt.ylabel("Précision")
plt.title("Performance du modèle")
plt.legend()
plt.show()